In [1]:
import datasets
import json
import os
import numpy as np
import collections
import re
import string
import math

In [2]:
# Change these variables to perform evalution for you task
test_path = os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'processed_data/gpt3/completion_6/processed_test.json'
quiz_path = os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_6/generated_quiz.json'
generated_path = os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_6/generated_answers_macaw.json'

In [3]:
# Get all the target prompts and completions
test_data = []

for line in open(test_path):
    test_data.append((json.loads(line)))

# Get quizzes
quizzes = []
with open(quiz_path) as f:
    generated_quizzes = json.load(f)

for key in generated_quizzes:
    quizzes.append(generated_quizzes[key])
    
# Get the generated completions
generated_data = []
with open(generated_path) as f:
    generated = json.load(f)

for key in generated:
    generated_data.append(generated[key])

In [4]:
count = 0
similar_count = 0

predictions = []
gold_references = []

for i in range(len(test_data)):
    count += 1
    
    completion = quizzes[i].split('True answer: ')[1].split("\nFalse answer")[0]
    clean_completion = completion.replace(" ","").replace("~","")
    
    generated_completion = generated_data[i][0].split('= ')[-1]
    clean_generated_completion = generated_completion.replace(" ","")
    
    if clean_completion == clean_generated_completion:
        similar_count += 1
        
    predictions.append(generated_completion)
    gold_references.append(completion)
    
print('Accuracy:', round(similar_count/count * 100, 2), '%')

Accuracy: 70.53 %


In [5]:
# Create result files

with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_6/' + 'predictions_answers_macaw.txt', 'w') as f:
    for i in range(len(predictions)):
        f.write(predictions[i] + '\n')
        
with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_6/' + 'ground_truth_answers.txt', 'w') as f:
    for i in range(len(gold_references)):
        f.write(gold_references[i] + '\n')
        
with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_6/' + 'automatic_evaluation_macaw.txt', 'w') as f:
    f.write('Accuracy: ' + str(round(similar_count/count * 100, 2)) + '%')

In [6]:
count = 0
with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'human_evaluation/experiment6/' + 'exp6_gpt3.txt', 'w') as f:
    for i in range(0,len(test_data),math.floor(len(test_data)/100)):
        f.write('Excel row: ' + str(count+2) + ' Test instance: ' + str(i+1) + '\n\n')
        f.write(test_data[i]['prompt'].split("\n\n###")[0] + '\n\n')
        f.write('Generated quiz:\n')
        f.write(quizzes[i].strip() + '\n\n')
        f.write('----------------------------------------------------------------------------------------' + '\n\n')
        count+=1
        
        if count == 100:
            break